In [1]:
!pip install neat-python

In [2]:
!pip install pygame

In [4]:
import neat

config_path = r"C:\Users\hiran\carsimulation\config-feedforward3.txt"  # Path to your config file
config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path
)

In [5]:
import pygame
import math
import sys
import neat
from IPython.display import display, clear_output

# Screen size and generation counter
screen_width = 1500
screen_height = 800
generation = 0

class Car:
    def __init__(self):
        # Load the car image and scale it
        try:
            self.surface = pygame.image.load(r"C:\Users\hiran\carsimulation\car.png")  # Ensure the image exists in the correct path
            self.surface = pygame.transform.scale(self.surface, (100, 100))
        except pygame.error:
            print("Error loading car.png. Make sure the file exists.")
            sys.exit(1)

        self.rotate_surface = self.surface
        self.pos = [700, 650]  # Initial position
        self.angle = 0
        self.speed = 0
        self.center = [self.pos[0] + 50, self.pos[1] + 50]
        self.radars = []
        self.radars_for_draw = []
        self.is_alive = True
        self.goal = False
        self.distance = 0
        self.time_spent = 0

    def draw(self, screen):
        # Draw the car and its radars
        screen.blit(self.rotate_surface, self.pos)
        self.draw_radar(screen)

    def draw_radar(self, screen):
        for r in self.radars:
            pos, dist = r
            pygame.draw.line(screen, (0, 255, 0), self.center, pos, 1)
            pygame.draw.circle(screen, (0, 255, 0), pos, 5)

    def check_collision(self, map):
        self.is_alive = True
        for p in self.four_points:
            if map.get_at((int(p[0]), int(p[1]))) == (255, 255, 255, 255):  # White pixels as obstacles
                self.is_alive = False
                break

    def check_radar(self, degree, map):
        length = 0
        x = int(self.center[0] + math.cos(math.radians(360 - (self.angle + degree))) * length)
        y = int(self.center[1] + math.sin(math.radians(360 - (self.angle + degree))) * length)

        while not map.get_at((x, y)) == (255, 255, 255, 255) and length < 300:
            length = length + 1
            x = int(self.center[0] + math.cos(math.radians(360 - (self.angle + degree))) * length)
            y = int(self.center[1] + math.sin(math.radians(360 - (self.angle + degree))) * length)

        dist = int(math.sqrt(math.pow(x - self.center[0], 2) + math.pow(y - self.center[1], 2)))
        self.radars.append([(x, y), dist])

    def update(self, map):
        # Update the car's movement and check boundaries
        self.speed = 15

        # Update car position
        self.rotate_surface = self.rot_center(self.surface, self.angle)
        self.pos[0] += math.cos(math.radians(360 - self.angle)) * self.speed
        if self.pos[0] < 20:
            self.pos[0] = 20
        elif self.pos[0] > screen_width - 120:
            self.pos[0] = screen_width - 120

        self.distance += self.speed
        self.time_spent += 1
        self.pos[1] += math.sin(math.radians(360 - self.angle)) * self.speed
        if self.pos[1] < 20:
            self.pos[1] = 20
        elif self.pos[1] > screen_height - 120:
            self.pos[1] = screen_height - 120

        # Calculate collision points
        self.center = [int(self.pos[0]) + 50, int(self.pos[1]) + 50]
        length = 40
        left_top = [self.center[0] + math.cos(math.radians(360 - (self.angle + 30))) * length, self.center[1] + math.sin(math.radians(360 - (self.angle + 30))) * length]
        right_top = [self.center[0] + math.cos(math.radians(360 - (self.angle + 150))) * length, self.center[1] + math.sin(math.radians(360 - (self.angle + 150))) * length]
        left_bottom = [self.center[0] + math.cos(math.radians(360 - (self.angle + 210))) * length, self.center[1] + math.sin(math.radians(360 - (self.angle + 210))) * length]
        right_bottom = [self.center[0] + math.cos(math.radians(360 - (self.angle + 330))) * length, self.center[1] + math.sin(math.radians(360 - (self.angle + 330))) * length]
        self.four_points = [left_top, right_top, left_bottom, right_bottom]
        
        self.check_collision(map)
        self.radars.clear()
        for d in range(-90, 120, 45):
            self.check_radar(d, map)

    def get_data(self):
        radars = self.radars
        ret = [0, 0, 0, 0, 0]
        for i, r in enumerate(radars):
            ret[i] = int(r[1] / 30)
        return ret

    def get_alive(self):
        return self.is_alive

    def get_reward(self):
        return self.distance / 50.0 + self.time_spent / 1000.0

    def rot_center(self, image, angle):
        orig_rect = image.get_rect()
        rot_image = pygame.transform.rotate(image, angle)
        rot_rect = orig_rect.copy()
        rot_rect.center = rot_image.get_rect().center
        rot_image = rot_image.subsurface(rot_rect).copy()
        return rot_image

def run_car(genomes, config):
    nets = []
    cars = []

    for id, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        g.fitness = 0
        cars.append(Car())  # Init cars

    pygame.init()
    screen = pygame.display.set_mode((screen_width, screen_height))
    pygame.display.set_caption("Car Running Simulation")
    clock = pygame.time.Clock()

    # Try to load the map image (if it fails, use a plain background)
    try:
        map = pygame.image.load(r"C:\Users\hiran\carsimulation\map.png")  # Ensure the map image exists at this path
    except pygame.error:
        print("Error loading map.png. Using plain background.")
        map = pygame.Surface((screen_width, screen_height))
        map.fill((0, 0, 0))  # Fill the background with black

    global generation
    generation += 1
    max_time = 1000  # Max time per generation

    # Main simulation loop
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

        # Input from car sensors and result from the neural network
        for index, car in enumerate(cars):
            output = nets[index].activate(car.get_data())
            i = output.index(max(output))
            if i == 0:
                car.angle += 10
            else:
                car.angle -= 10

        # Update cars and fitness
        remain_cars = 0
        for i, car in enumerate(cars):
            if car.get_alive():
                remain_cars += 1
                car.update(map)
                genomes[i][1].fitness += car.get_reward()

        # Check for termination conditions (e.g., all cars are dead or max time reached)
        if remain_cars == 0 or generation >= max_time:
            break

        # Drawing
        screen.blit(map, (0, 0))
        for car in cars:
            if car.get_alive():
                car.draw(screen)

        # Display generation and remaining cars
        generation_font = pygame.font.SysFont("Arial", 70)
        font = pygame.font.SysFont("Arial", 30)

        text = generation_font.render("Generation : " + str(generation), True, (255, 255, 0))
        text_rect = text.get_rect()
        text_rect.center = (screen_width / 2, 100)
        screen.blit(text, text_rect)

        text = font.render("Remain cars: " + str(remain_cars), True, (0, 0, 0))
        text_rect = text.get_rect()
        text_rect.center = (screen_width / 2, 200)
        screen.blit(text, text_rect)

        pygame.display.update()  # Non-blocking update

        clock.tick(360)  # Set FPS to 30 for smooth performance

# Run the NEAT simulation
if __name__ == "__main__":
    config_path = r"C:\Users\hiran\carsimulation\config-feedforward3.txt"  # Ensure the config file path is correct
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                                neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

    # Create the core NEAT population
    p = neat.Population(config)

    # Add reporters
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)

    # Run the NEAT simulation
    p.run(run_car, 10)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html

 ****** Running generation 0 ****** 

Population's average fitness: 4.72570 stdev: 4.47479
Best fitness: 16.55500 - size: (2, 10) - species 1 - id 2
Average adjusted fitness: 0.244
Mean genetic distance 1.232, standard deviation 0.454
Population of 10 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    10     16.6    0.244     0
Total extinctions: 0
Generation time: 1.447 sec

 ****** Running generation 1 ****** 

Population's average fitness: 7.97650 stdev: 5.58961
Best fitness: 16.55500 - size: (2, 10) - species 1 - id 2
Average adjusted fitness: 0.452
Mean genetic distance 0.960, standard deviation 0.251
Population of 10 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    10     16.6    0.452     1
Total extinctions: 0
Generation time: